In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from ast import literal_eval

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [25]:
THEMES = [5, 6, 26, 33, 139, 163, 232, 313, 339, 350, 406, 409, 555, 589,
          597, 634, 660, 695, 729, 766, 773, 793, 800, 810, 852, 895, 951, 975]
TRAIN_DATA_PATH = '../train_small.csv'
TEST_DATA_PATH = '../test_small.csv'
VALIDATION_DATA_PATH = '../validation_small.csv'

In [29]:
def get_data(path, preds=None, key=None):
    data = pd.read_csv(path)
    data = data.rename(columns={ 'pages': 'page'})
    data.body = data.body.str.strip('{}"')
    data = groupby_process(data)
    data.themes = data.themes.apply(lambda x: literal_eval(x))
    return data

In [30]:
def groupby_process(df):
    new_df = df.sort_values(['process_id', 'page'])
    new_df = new_df.groupby(
                ['process_id', 'themes'],
                group_keys=False
            ).apply(lambda x: x.body.str.cat(sep=' ')).reset_index()
    new_df = new_df.rename(index=str, columns={0: "body"})
    return new_df

In [31]:
train_data = get_data(TRAIN_DATA_PATH)
test_data = get_data(TEST_DATA_PATH)
validation_data = get_data(VALIDATION_DATA_PATH)

In [32]:
train_data.themes = train_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
test_data.themes = test_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
validation_data.themes = validation_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))

In [33]:
len(train_data), len(validation_data), len(test_data)

(2743, 1893, 1874)

In [34]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(sentence.split(" "))

train_words = list(sent_to_words(train_data.body.tolist()))


print(train_words[:1])

[['tribunal', 'justiça', 'estado', 'bahia', 'poder', 'judiciário', 'salvador', 'juizado', 'cível', 'defesa', 'consumidor', 'universo', 'matutino', 'projudi', 'acm', 'campus', 'universidade', 'salgado', 'oliveira', 'universo', 'pituba', 'salvador', 'processo', 'parte', 'autora', 'nailton', 'lantyer', 'cordeiro', 'araujo', 'parte', 'atlantico', 'fundo', 'investimento', 'direitos', 'creditorios', 'despacho', 'vistos', 'etc', 'indefiro', 'requerido', 'pela', 'parte', 'demandante', 'aguarde', 'a', 'sessão', 'conciliação', 'designada', 'salvador', 'de', 'julho', 'paulo', 'alberto', 'nunes', 'chenaud', 'juiz', 'direito', 'documento', 'assinado', 'eletronicamente', 'excelentíssimo', 'senhor', 'doutor', 'juiz', 'direito', 'juizado', 'cível', 'defesa', 'consumidor', 'universo', 'comarca', 'salvador', 'autos', 'processo', 'º', 'atlântico', 'fundo', 'investimento', 'direitos', 'creditórios', 'não', 'padronizados', 'nova', 'denominação', 'crdg', 'fundo', 'investimento', 'direitos', 'creditórios', '

In [35]:
len(train_words)

2743

In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(train_words, min_count=50, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[train_words], min_count=5, threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[train_words[0]]])

['tribunal', 'justiça', 'estado', 'bahia', 'poder', 'judiciário', 'salvador', 'juizado', 'cível', 'defesa_consumidor', 'universo', 'matutino_projudi', 'acm', 'campus', 'universidade', 'salgado', 'oliveira', 'universo', 'pituba', 'salvador', 'processo', 'parte', 'autora', 'nailton_lantyer', 'cordeiro', 'araujo', 'parte', 'atlantico_fundo', 'investimento', 'direitos', 'creditorios', 'despacho', 'vistos_etc', 'indefiro', 'requerido', 'pela', 'parte', 'demandante', 'aguarde', 'a', 'sessão', 'conciliação', 'designada', 'salvador', 'de', 'julho', 'paulo', 'alberto', 'nunes', 'chenaud', 'juiz', 'direito', 'documento', 'assinado', 'eletronicamente', 'excelentíssimo_senhor_doutor', 'juiz', 'direito', 'juizado', 'cível', 'defesa_consumidor', 'universo', 'comarca', 'salvador', 'autos', 'processo', 'º', 'atlântico_fundo', 'investimento', 'direitos_creditórios', 'não', 'padronizados', 'nova', 'denominação', 'crdg', 'fundo_investimento', 'direitos_creditórios', 'não', 'padronizados', 'empresa', 'ins

In [38]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [39]:
train_words = make_bigrams(train_words)
train_words = make_trigrams(train_words)

In [111]:
# Create Dictionary
id2word = corpora.Dictionary(train_words)

In [42]:
def save_dic(dic, filename="dic"):
    with open(filename, "wb") as handle:
        dic.save(handle)

In [43]:
save_dic(id2word, "small_dict")

In [112]:
len(id2word.dfs), id2word.dfs

(221049,
 {2415: 2740,
  1409: 2072,
  949: 2050,
  270: 188,
  1810: 1240,
  1378: 1924,
  2187: 139,
  1380: 1165,
  625: 1187,
  669: 97,
  2436: 159,
  1502: 2,
  58: 3,
  326: 12,
  2435: 109,
  2182: 42,
  1684: 1104,
  1799: 1,
  1916: 2707,
  1733: 1827,
  258: 1169,
  1578: 1,
  574: 117,
  196: 312,
  235: 2,
  1349: 69,
  788: 1042,
  586: 2,
  727: 2268,
  2502: 261,
  1255: 249,
  2119: 708,
  1762: 1635,
  687: 574,
  113: 129,
  31: 2453,
  2242: 1056,
  468: 198,
  720: 131,
  649: 2741,
  1393: 1204,
  1752: 1682,
  116: 514,
  1641: 375,
  381: 1,
  1379: 1210,
  787: 2347,
  813: 2552,
  218: 2416,
  865: 543,
  987: 841,
  425: 232,
  263: 2699,
  2527: 1089,
  237: 2,
  789: 5,
  1644: 1945,
  1721: 20,
  1635: 1083,
  698: 107,
  585: 1,
  1130: 16,
  881: 412,
  1309: 447,
  400: 529,
  2262: 2553,
  424: 2589,
  2202: 997,
  265: 184,
  302: 12,
  1040: 316,
  1445: 667,
  145: 737,
  2325: 1864,
  1819: 2369,
  2245: 1187,
  94: 532,
  2251: 15,
  1630: 2496,
 

In [155]:
from copy import deepcopy

copy_dict = deepcopy(id2word)
copy_dict.filter_extremes(no_below=5, no_above=.5, keep_n=None)

In [156]:
len(copy_dict.dfs), copy_dict.dfs

(55794,
 {237: 188,
  1585: 1240,
  1929: 139,
  1220: 1165,
  558: 1187,
  595: 97,
  2143: 159,
  288: 12,
  2142: 109,
  1925: 42,
  1476: 1104,
  226: 1169,
  514: 117,
  170: 312,
  1194: 69,
  697: 1042,
  2204: 261,
  1108: 249,
  1865: 708,
  612: 574,
  96: 129,
  1977: 1056,
  412: 198,
  641: 131,
  1231: 1204,
  99: 514,
  1442: 375,
  1219: 1210,
  765: 543,
  869: 841,
  373: 232,
  2228: 1089,
  698: 5,
  1509: 20,
  1436: 1083,
  623: 107,
  1000: 16,
  778: 412,
  1157: 447,
  355: 529,
  1942: 997,
  232: 184,
  266: 12,
  915: 316,
  1276: 667,
  126: 737,
  1979: 1187,
  80: 532,
  1983: 15,
  235: 1226,
  811: 1203,
  1704: 898,
  2166: 514,
  870: 697,
  1101: 524,
  1767: 1202,
  967: 923,
  992: 821,
  18: 457,
  1260: 1328,
  1185: 1041,
  1155: 1019,
  1788: 943,
  745: 1148,
  2042: 12,
  473: 198,
  1774: 1183,
  128: 523,
  19: 169,
  1524: 953,
  27: 828,
  1799: 1202,
  1142: 861,
  1641: 318,
  1976: 1166,
  53: 87,
  1048: 731,
  1947: 1203,
  1186: 103

In [157]:
id2word = deepcopy(copy_dict)
del(copy_dict)

In [158]:
[(id2word[x], y) for (x, y) in sorted(id2word.dfs.items(), key=lambda x: x[1], reverse=True)]

[('automaticamente', 1369),
 ('atual', 1367),
 ('advogado', 1364),
 ('sua', 1363),
 ('dada', 1363),
 ('regimento_interno', 1362),
 ('termo_remessa_externa', 1362),
 ('faço_remessa_destes', 1362),
 ('determino_devolução_destes', 1361),
 ('contra', 1360),
 ('institui_infra_estrutura_chaves', 1359),
 ('peças_processuais', 1359),
 ('instituto', 1358),
 ('outubro', 1356),
 ('nesta', 1356),
 ('públicas_brasileira_icp_brasil', 1355),
 ('caput', 1350),
 ('prazo', 1350),
 ('anterior', 1343),
 ('ministra_cármen_lúcia', 1343),
 ('concessão', 1330),
 ('lei', 1328),
 ('base', 1326),
 ('seja', 1325),
 ('devido', 1325),
 ('deste', 1324),
 ('setembro', 1324),
 ('considerando', 1323),
 ('inc', 1323),
 ('aposentadoria', 1323),
 ('legais', 1319),
 ('juizados', 1319),
 ('cpc', 1318),
 ('abril', 1318),
 ('interposto', 1315),
 ('maio', 1315),
 ('ciência', 1312),
 ('caso', 1310),
 ('procedimento', 1308),
 ('remessa', 1306),
 ('após', 1303),
 ('baixa', 1303),
 ('março', 1299),
 ('esse', 1295),
 ('seguro', 129

In [159]:
save_dic(id2word, "small_dict")

In [160]:
train_corpus = [id2word.doc2bow(text) for text in train_words]

In [161]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [162]:
pprint(lda_model.print_topics())

[(0,
  '0.023*"institui_infra_estrutura_chaves" + '
  '0.023*"públicas_brasileira_icp_brasil" + 0.019*"outubro" + '
  '0.015*"senhor_secretário" + 0.012*"ministra_cármen_lúcia" + '
  '0.012*"peças_processuais" + 0.012*"determino_devolução_destes" + '
  '0.011*"execução" + 0.011*"termo_remessa_externa" + '
  '0.011*"faço_remessa_destes"'),
 (1,
  '0.026*"r" + 0.025*"d" + 0.020*"c" + 0.020*"i" + 0.019*"união" + 0.018*"u" + '
  '0.015*"t" + 0.014*"fls" + 0.012*"advogado" + 0.012*"b"'),
 (2,
  '0.024*"fazenda" + 0.020*"imposto" + 0.020*"renda" + 0.016*"contribuição" + '
  '0.013*"incidência" + 0.012*"servidor" + 0.010*"base" + 0.010*"lei" + '
  '0.009*"gacen" + 0.008*"procuradoria"'),
 (3,
  '0.020*"servidores" + 0.015*"gratificação" + 0.013*"união" + '
  '0.012*"desempenho" + 0.011*"saúde" + 0.010*"servidor" + 0.009*"lei" + '
  '0.008*"atividade" + 0.008*"pública" + 0.006*"gdpst"'),
 (4,
  '0.005*"valor" + 0.005*"caso" + 0.004*"poder" + 0.004*"sua" + 0.004*"uma" + '
  '0.004*"união" + 0.0

In [163]:
lda_model.save("lda_small_first")

/home/isis/.env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/isis/.env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [164]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, train_corpus, id2word)
vis

/home/isis/.env/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.188138  0.019802       1        1  23.206047
4      0.157445  0.037060       2        1  21.332111
2      0.189892 -0.052961       3        1  14.425117
3      0.186696  0.022815       4        1  13.846485
8      0.184713  0.040580       5        1   9.889957
5     -0.199383  0.142794       6        1   9.315334
6      0.031530 -0.130747       7        1   3.109593
7     -0.248917 -0.361366       8        1   2.224164
1     -0.238199  0.177195       9        1   1.825053
0     -0.251916  0.104829      10        1   0.826147, topic_info=      Category          Freq                             Term         Total  \
term                                                                          
2156   Default  92956.000000                            valor  92956.000000   
8448   Default  51020.000000                             upag  51020.000000   
854    Default  42634.000000                           evento  42634.000000   
22358  Default  27895.000000                              ipi  27895.000000   
1170   Default  55272.000000                        instituto  55272.000000   
554    Default  53323.000000                          cálculo  53323.000000   
1078   Default  50072.000000                          imposto  50072.000000   
4669   Default  53134.000000                           seguro  53134.000000   
244    Default  49110.000000                             base  49110.000000   
4948   Default  65574.000000                            união  65574.000000   
3421   Default  62084.000000                          fazenda  62084.000000   
79     Default  45230.000000                         advogado  45230.000000   
4681   Default  48670.000000                       servidores  48670.000000   
4536   Default  71026.000000                            renda  71026.000000   
2653   Default  83974.000000                        benefício  83974.000000   
8994   Default  28511.000000                           previd  28511.000000   
1316   Default  39079.000000                            maria  39079.000000   
1767   Default  23170.000000                                r  23170.000000   
329    Default  28666.000000                              cep  28666.000000   
1934   Default  31091.000000                           santos  31091.000000   
3719   Default  34422.000000                             inss  34422.000000   
2104   Default  20219.000000                            total  20219.000000   
3536   Default  40765.000000                     gratificação  40765.000000   
2895   Default  28072.000000                    contribuições  28072.000000   
8097   Default  15715.000000                          usuário  15715.000000   
970    Default  21732.000000                             fone  21732.000000   
4318   Default  35237.000000                   previdenciário  35237.000000   
4320   Default  52108.000000                      previdência  52108.000000   
4637   Default  20305.000000                              rua  20305.000000   
2068   Default  42534.000000                            tempo  42534.000000   
...        ...           ...                              ...           ...   
5945   Topic10   2826.478027   públicas_brasileira_icp_brasil   3421.346680   
5929   Topic10   2858.823242  institui_infra_estrutura_chaves   3512.632324   
5899   Topic10   1101.262939              volumes_qtd_apensos   1263.527100   
5881   Topic10   1115.454712         foram_recebidos_autuados   1298.269287   
5893   Topic10   1115.396118                   qtd_folhas_qtd   1298.214966   
5940   Topic10   1457.390869                peças_processuais   1751.027100   
5963   Topic10    591.219360                     dados_folhas    660.892822   
5887   Topic10   1476.627930            ministra_cármen_lúcia   1906.400024   
5868   Topic10    681.935669                      ARTIGO_1036    788.7

In [ ]:
import os
os.environ.update({'MALLET_HOME':"/home/isis/Davi_Alves/data/parts/topic_modeling/mallet/mallet-2.0.8"})

mallet_path = "/home/isis/Davi_Alves/data/parts/topic_modeling/mallet/mallet-2.0.8/bin/mallet" # update this path

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=train_corpus, num_topics=10, id2word=id2word, workers=10)

/home/isis/.env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
pprint(ldamallet.print_topics())

In [ ]:
ldamallet.save("/models/ldamallet_small")